In [1]:
import sys
sys.path.append('..')
sys.path.append('../..')

import pandas as pd
from IPython.display import display


from src.tools import *
from src.database import Database
from src.accountant import Accountant
from src.prices.prices import Price

p = Price()
price = p.price
db = Database('db.db')
act = Accountant(db)
db1 = Database('db1.db')

txs = db1.read('valorized')

leviamo le transazioni interne

In [2]:
txs = txs[txs['label1']!='internal']

elimino a mano le transazioni di token problematici (la cui contabilità va eseguita a parte), se è il caso

In [3]:
# def deleteToken(df, token):
#     assert token in db.coins(), 'se accade questo, ricontrollare'
#     mask = (df['in_iso'] == token) | (df['out_iso']== token) | (df['fee_iso']==token)
#     return df[~mask]
# txs = deleteToken(txs, 'TOKEN SGRADITO')

creo le posizioni aperte a partire dalle transazioni "valorized"

In [3]:
col_open = ['open_id', 'open_t', 'open_amount','open_price','coin', 'amount']
col_close = ['close_id', 'close_t', 'close_price', 'close_amount', 'cgain']

In [4]:
def open_pos(r):
    out = {
        'open_id': r['id'],
        'open_t':r['timestamp'],
        'open_amount':r['in'],
        'open_price': r['in_price'],
        'coin': r['in_iso'],
        'amount':r['in']
    }
    if r['type']!='trade':
        out['open_price'] = 0
    return pd.Series(out)

In [5]:
pos = txs.apply(open_pos, axis=1)
pos=pos[pos['amount'].notna()]
pos[col_close] = None
len(pos)

271

aggiungo le posizioni che corrispondono ai bilanci del momento iniziale (tutte a costo di carico 0)

In [6]:
start_time = '2017-01-01 00:00:00'
starting = act.shot(start_time)['global'].to_dict()

In [7]:
startt = timeFormat(start_time)
lll = []
for i, c in enumerate(starting):
    lll.append({
        'open_id':'start'+str(i),
        'open_t': startt,
        'open_amount':starting[c],
        'open_price':0,
        'coin':c,
        'amount':starting[c]
    })

pos = pd.concat([pos, pd.DataFrame(lll)], ignore_index=True)
pos.sort_values(by='open_t', inplace=True)

aggiungo a mano transazioni specifiche per aggiustare la contabilità ove necessario, per es:
- commissioni non contabilizzare
- transazioni particolari, specifiche dei recipients
  

In [10]:
# llll = []
# adjust_id = 'adjust'
# adjust_time = '9999-99-99 99:99:99'
# llll.append({
#     'open_t': timeFormat(adjust_time),
#     'open_id': adjust_id+str(timeFormat(adjust_time)),
#     'open_amount':0.02,
#     'open_price':0,
#     'coin':'BNB',
#     'amount':0.02
# })


# pos = pd.concat([pos, pd.DataFrame(llll)], ignore_index=True)
# pos.sort_values(by='open_t', inplace=True)

In [8]:
# un print da attivare solo per il debug
verbose = False
def stampa(*args):
    global verbose
    if (verbose): print(*args)

In [9]:
def find_open(iso, timestamp):
    #stampa('find_open', iso, timestamp)
    global pos
    mask = (pos['coin']==iso ) & (pos['open_t']<=timestamp) & pos['close_t'].isna() 
    tmp = pos[mask]
    tmp = tmp.sort_values(by='open_t', ascending=False)
    if verbose: display(tmp)
    return tmp.index[0]

In [10]:
threshold = 0.00001 # €

def close_pos(id, timestamp, coin, amount, p):
    global pos
    global verbose
    global threshold
    
    if pd.isna(p) or (p == 0): 
        #verbose = True
        stampa('OCCHIO CHIUSURA A PREZZO ZERO, prezzo:',p)
        p = 'unknown'
    else:
        verbose = False
    
    # verbose = True if coin == 'MAID' else False

    if p != 'unknown' and p*amount < threshold: return 0
    
     

    try:
        stampa('########################  close', coin, amount,'(',id,  p, timestamp, ')')
        id_open = find_open(coin,timestamp)
    except Exception as e:
        verbose = True
        stampa('EXCEPTION WHILE CLOSING', id, timestamp, coin, amount, p)
        id_open = find_open(coin,timestamp)


    openpos = pos.loc[id_open].to_dict()
    closepos = pos.loc[id_open].to_dict()

    #stampa('open position:', openpos)

    assert closepos['coin'] == coin

    closepos['close_id'] = id
    closepos['close_t'] = timestamp
    closepos['close_price'] = p

    if amount >= openpos['amount']:
        close_amount = openpos['amount']
    else:
        close_amount = amount
    
    closepos['close_amount'] = close_amount
    if p == 'unknown' : closepos['cgain']=p
    else: closepos['cgain'] = close_amount * (closepos['close_price']- closepos['open_price'])

    stampa('infine:')

    if amount >= openpos['amount']:
        stampa('chiudi tutta')
        rest = amount - openpos['amount']
        stampa(closepos)
        stampa('rest',rest)
        # chiudila tutta e avanza il resto:
        pos.loc[id_open] = pd.Series(closepos)

        if p == 'unknown': return rest
        else:
            rest_value = rest * p
            return rest if rest_value > threshold else 0
        
    else:
        stampa('lascia aperta')
        # chiudine una parte e crea una nuova pos aperta, returns resto 0
        pos.loc[id_open] = pd.Series(closepos)

        rest = openpos['amount'] - amount
        openpos['amount'] = rest

        if p != 'unknown': 
            rest_value = rest*p
            stampa('rest', rest_value, 'threshold', threshold)
            if rest_value > threshold:
                pos = pd.concat([pos, pd.DataFrame([openpos])], ignore_index=True)
                stampa('left:', openpos)
            else:          
                stampa('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
                stampa('left:', openpos)
                stampa('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
        else:
            pos = pd.concat([pos, pd.DataFrame([openpos])], ignore_index=True)
            stampa('left:', openpos)

    
         
        stampa('closed:',closepos)
        return 0


In [11]:
alltxs = db1.read('valorized')
#alltxs = deleteToken(alltxs, 'TOKEN SGRADITO')

for i, r in alltxs.iterrows():
    if r['label1'] != 'internal':
        if pd.notna(r['out']):
            rest = r['out']
            while rest>0:
                rest =  close_pos(r['id'], r['timestamp'], r['out_iso'], rest, r['out_price'])       
    if pd.notna(r['fee']):
        rest = r['fee']
        while rest>0:
            rest = close_pos(r['id'], r['timestamp'], r['fee_iso'],rest, r['fee_price'])

    

In [12]:
db1.save('positions', pos)